# Utility Chains

In [1]:
import os
os.environ["OPENAI_API_KEY"] = ""

In [2]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains.summarize import load_summarize_chain
from langchain.text_splitter import CharacterTextSplitter
from langchain.docstore.document import Document

In [3]:
llm = OpenAI(temperature=0.9)

/Users/venkatasamyuktamalapaka/anaconda3/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.openai.OpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


In [4]:
with open('/Users/venkatasamyuktamalapaka/Downloads/Data Connections - Python Code/Sample.txt') as d:
    data = d.read()  

In [5]:
text_splitter = CharacterTextSplitter()
texts = text_splitter.split_text(data)

In [6]:
docs = [Document(page_content=t) for t in texts]
docs

[Document(page_content="India, officially known as the Republic of India, is a diverse and vibrant country located in South Asia. With a rich history spanning thousands of years, India is known for its cultural heritage, religious diversity, and vast landscapes. From the majestic Himalayas in the north to the serene backwaters of Kerala in the south, India encompasses a wide range of geographical features, including deserts, plains, mountains, and coastlines, making it a land of incredible natural beauty.\n\nIndia is the seventh-largest country by land area and the second-most populous country in the world, with a population exceeding 1.3 billion people. It is a federal parliamentary democratic republic, with a president as the head of state and a prime minister as the head of government. The country follows a multi-tiered administrative structure, with 28 states and 9 union territories, each having its own elected government.\n\nIndia has a rich cultural heritage that has evolved over

In [8]:
chain = load_summarize_chain(llm, chain_type = "map_reduce", verbose = True)

In [9]:
chain.run(docs)

/Users/venkatasamyuktamalapaka/anaconda3/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new MapReduceDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Write a concise summary of the following:


"India, officially known as the Republic of India, is a diverse and vibrant country located in South Asia. With a rich history spanning thousands of years, India is known for its cultural heritage, religious diversity, and vast landscapes. From the majestic Himalayas in the north to the serene backwaters of Kerala in the south, India encompasses a wide range of geographical features, including deserts, plains, mountains, and coastlines, making it a land of incredible natural beauty.

India is the seventh-largest country by land area and the second-most populous country in the world, with a population exceeding 1.3 billion people. It is a federal parliamentary democratic republic, with a president as the head of state and a prime minister as the head of government. The country follows a multi-tiered administrative structure, with 28 st

' India is a diverse and vibrant country in South Asia, known for its cultural heritage, religious diversity, and fast-growing economy. It has a federal parliamentary democratic republic government and a significant influence globally in fields such as STEM, film, and cuisine. The Indian rupee is its official currency, controlled by the Reserve Bank of India. The country also values education and has a strong presence in tourism, cultural heritage, space exploration, and diplomacy. '

# HTTP Requests

# LLMRequests Chain

In [11]:
from langchain.chains import LLMRequestsChain, LLMChain

In [15]:
template = """
Extract answers to the question '{query}' or say "not found" if the information is not available
{requests_result}
"""

PROMPT = PromptTemplate(
    input_variables=["query","requests_result"],
    template = template
)

In [16]:
llm = OpenAI()

In [17]:
chain = LLMRequestsChain(llm_chain = LLMChain(llm=llm,prompt=PROMPT))

In [18]:
question = "What is the capital of India?"
inputs = {
    "query":question,
    "url":"https://www.google.com/search?q=" + question.replace(" ", "+"),
}

In [19]:
chain(inputs)

/Users/venkatasamyuktamalapaka/anaconda3/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


{'query': 'What is the capital of India?',
 'url': 'https://www.google.com/search?q=What+is+the+capital+of+India?',
 'output': '\nThe capital of India is New Delhi.'}

In [20]:
import inspect
print(inspect.getsource(chain._call))

    def _call(
        self,
        inputs: Dict[str, Any],
        run_manager: Optional[CallbackManagerForChainRun] = None,
    ) -> Dict[str, Any]:
        from bs4 import BeautifulSoup

        _run_manager = run_manager or CallbackManagerForChainRun.get_noop_manager()
        # Other keys are assumed to be needed for LLM prediction
        other_keys = {k: v for k, v in inputs.items() if k != self.input_key}
        url = inputs[self.input_key]
        res = self.requests_wrapper.get(url)
        # extract the text from the html
        soup = BeautifulSoup(res, "html.parser")
        other_keys[self.requests_key] = soup.get_text()[: self.text_length]
        result = self.llm_chain.predict(
            callbacks=_run_manager.get_child(), **other_keys
        )
        return {self.output_key: result}

